Notebook Setup

In [1]:
#import dependencies
import requests
from pprint import pprint
from requests.auth import HTTPBasicAuth
import pandas as pd

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import scipy.stats as stats

#authentication flow
client_id = "4b454f446d1f479082eec9de1c27a4a8"
client_secret = "e639450422594948b17c46d1a82f4882"

url = "https://accounts.spotify.com/api/token?grant_type=client_credentials"
headers = {'content-type': 'application/x-www-form-urlencoded'}

try:
   response = requests.post(url, headers=headers, auth=(client_id, client_secret))
   json = response.json()
   print(json)
   access_token = json["access_token"]
   print(access_token)
except:
   print("Something went wrong with the request")

{'access_token': 'BQCuJ2DITuH2NEzvrICjyzOoBrDqB4XCLVIPCZW4ATVCutOy8Eu_ESnATxcNPxHrjwh8c71RU4uCAt--DQ0', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}
BQCuJ2DITuH2NEzvrICjyzOoBrDqB4XCLVIPCZW4ATVCutOy8Eu_ESnATxcNPxHrjwh8c71RU4uCAt--DQ0


Collect Data

In [2]:
# https://api.spotify.com/v1/artists/{id}

#This is for US Top 50 playlist

playlist_id = "37i9dQZEVXbMXbN3EUUhlg"
#spotify:user:spotifycharts:playlist:37i9dQZEVXbLRQDuF5jeBp

url = f"https://api.spotify.com/v1/playlists/{playlist_id}"

headers = {'authorization': f"Bearer {access_token}"}
response = requests.get(url, headers=headers)

#pprint(response.json())
#print(json.dumps(newrelease, sort_keys=True, indent=4))
jj = response.json()
#pprint(jj)

In [3]:
"""for playlist in pl: 
    url = f"https://api.spotify.com/v1/playlists/{playlist}"

    headers = {'authorization': f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)

    #pprint(response.json())
    #print(json.dumps(newrelease, sort_keys=True, indent=4))
    jj = response.json()
    
    for x in jj:
        pprint(x['tracks'])"""

'for playlist in pl: \n    url = f"https://api.spotify.com/v1/playlists/{playlist}"\n\n    headers = {\'authorization\': f"Bearer {access_token}"}\n    response = requests.get(url, headers=headers)\n\n    #pprint(response.json())\n    #print(json.dumps(newrelease, sort_keys=True, indent=4))\n    jj = response.json()\n    \n    for x in jj:\n        pprint(x[\'tracks\'])'

In [4]:
#test check: to grab album uri
#jj['tracks']['items'][0]['track']['album']['uri'][14:])
#test check to grab artist uri
pprint(jj['tracks']['items'][0]['track']['album']['artists'][0]["uri"][15:])

'5bzWtCkjIAMgN93gLt56SO'


In [5]:
songs= jj['tracks']['items']

In [6]:
#Create lists for relevant data & then organize into a dataframe
ids = []
artists = []
tracks = []
releasedate = []
audiofeatures = []
popularity = []
for x in songs:
    idsong = x["track"]["id"]
    ids.append(idsong)
    artist = x['track']['album']['artists'][0]['name']
    artists.append(artist)
    track = x['track']['name']
    tracks.append(track)
    rd = x["track"]["album"]["release_date"]
    releasedate.append(rd)
    poppin = x['track']['popularity']
    popularity.append(poppin)
    url = f"https://api.spotify.com/v1/audio-features/{idsong}"
    headers = {'authorization': f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    features= response.json()
    audiofeatures.append(features)
#pprint(features)

In [7]:
#looking for genre of each artist via for loop
artid = []
genre = []
for x in songs:
    ida = x['track']['album']['artists'][0]["uri"][15:]
    artid.append(ida)
    url = f"https://api.spotify.com/v1/artists/{ida}"
    headers = {'authorization': f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    gen= response.json()
    genr = gen["genres"]
    genre.append(genr)

In [8]:
#separate the dictionary into lists
afkeys = [list(x.keys()) for x in audiofeatures]
afvalues = [list(x.values()) for x in audiofeatures]

#create a dataframe for audio features
af = pd.DataFrame(afvalues, columns=afkeys[0])
#create a dataframe for track info
tdata = pd.DataFrame(list(zip(artists, artid, tracks, ids, releasedate, popularity, genre)),columns=["artists", "artist id", "tracks", "track ids", "released", "popularity", "genre"])

In [9]:
tdata.head()

,artists,artist id,tracks,track ids,released,popularity,genre
0,Tropkillaz,5bzWtCkjIAMgN93gLt56SO,Bola Rebola,1OUPXna2MCgAt3VNmXJBtg,2019-02-22,77,"[baile pop, bass trap, electronic trap]"
1,Lady Gaga,1HY2Jd0NmPuamShAr6KMms,Shallow,2VxeLyX666F8uXCJ0dZF8B,2018-10-05,92,"[dance pop, pop]"
2,MC Kevin o Chris,2UMj7NCbuqy1yUZmiSYGjJ,Vamos pra Gaiola,7FiIQXpIeLo3ITfGGpb8PB,2019-01-11,81,[]
3,Marília Mendonça,1yR65psqiazQpeM79CcGh8,Bebi Liguei - Ao Vivo,3RbXsVQr4jeTxvT35Hssq3,2019-02-22,54,[sertanejo]
4,Felipe Araújo,47uyFQHOD02S0lj9ptRpoB,Atrasadinha - Ao Vivo,5EKHbW7mXcAR4hCyS3YXjb,2018-09-20,84,"[sertanejo pop, sertanejo universitario]"


In [10]:
#combine the two dataframes 
data = pd.concat([tdata, af], axis=1, sort=False)

#clean data by removing unneccesary columns
data.drop(["uri", "track_href", "analysis_url", "id"],axis = 1, inplace = True)
data

,artists,artist id,tracks,track ids,released,popularity,genre,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature
0,Tropkillaz,5bzWtCkjIAMgN93gLt56SO,Bola Rebola,1OUPXna2MCgAt3VNmXJBtg,2019-02-22,77,"[baile pop, bass trap, electronic trap]",0.834,0.777,5,...,0,0.0593,0.2200,0.006090,0.3350,0.865,129.984,audio_features,193440,4
1,Lady Gaga,1HY2Jd0NmPuamShAr6KMms,Shallow,2VxeLyX666F8uXCJ0dZF8B,2018-10-05,92,"[dance pop, pop]",0.572,0.385,7,...,1,0.0308,0.3710,0.000000,0.2310,0.323,95.799,audio_features,215733,4
2,MC Kevin o Chris,2UMj7NCbuqy1yUZmiSYGjJ,Vamos pra Gaiola,7FiIQXpIeLo3ITfGGpb8PB,2019-01-11,81,[],0.872,0.971,6,...,1,0.2810,0.9170,0.001430,0.1210,0.521,149.987,audio_features,161600,4
3,Marília Mendonça,1yR65psqiazQpeM79CcGh8,Bebi Liguei - Ao Vivo,3RbXsVQr4jeTxvT35Hssq3,2019-02-22,54,[sertanejo],0.603,0.804,0,...,1,0.1980,0.3620,0.000000,0.7430,0.682,153.883,audio_features,159780,4
4,Felipe Araújo,47uyFQHOD02S0lj9ptRpoB,Atrasadinha - Ao Vivo,5EKHbW7mXcAR4hCyS3YXjb,2018-09-20,84,"[sertanejo pop, sertanejo universitario]",0.676,0.844,7,...,1,0.1090,0.4980,0.000000,0.8660,0.716,159.969,audio_features,172767,4
5,Dennis DJ,6xlRSRMLgZbsSNd0BMobwy,Agora é Tudo Meu,5yH8pYxNckOU1cxfPsMIaz,2018-12-07,84,[baile funk],0.848,0.933,5,...,0,0.1060,0.7000,0.000003,0.1840,0.506,139.998,audio_features,171998,4
6,Anitta,7FNnA9vBm6EKceENgCGRMb,Terremoto,7G2XZhDNHo7SSDWSVxC0UT,2019-02-01,83,[baile pop],0.891,0.805,8,...,0,0.0898,0.3290,0.000000,0.2770,0.967,95.045,audio_features,146276,4
7,Mc Livinho,7me0S5Z40qVWj3gzyK8aC3,Hoje Eu Vou Parar Na Gaiola,0IQCOAw9ZAI6WnXIrl481F,2018-10-17,82,"[baile pop, deep funk carioca]",0.650,0.361,2,...,1,0.4040,0.0331,0.000000,0.0978,0.844,89.803,audio_features,177094,4
8,Zé Neto & Cristiano,487N2T9nIPEHrlTZLL3SQs,Notificação Preferida - Ao Vivo,2qkNnJvQvhLiZ63X1L3eFz,2018-05-18,80,[sertanejo universitario],0.398,0.838,8,...,0,0.1300,0.7780,0.000000,0.1890,0.206,156.008,audio_features,182341,3
9,Márcia Fellipe,3OxZ4asD88P1lJJmLcXz4K,Quem Me Dera,2lAyctg5FocS67eksHoyIg,2018-12-12,68,"[sertanejo pop, sertanejo universitario]",0.815,0.723,6,...,0,0.0574,0.1150,0.000000,0.1840,0.896,149.918,audio_features,147990,4


In [11]:
data.describe()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.00000,50.000000,50.00000
mean,76.680000,0.700500,0.734840,5.120000,-4.641560,0.580000,0.105172,0.350892,0.000153,0.345734,0.641140,128.41796,191505.300000,4.02000
std,10.033495,0.129531,0.164195,3.836665,2.066267,0.498569,0.083409,0.238452,0.000880,0.300191,0.211452,26.73292,62485.815218,0.24661
min,52.000000,0.398000,0.320000,0.000000,-10.752000,0.000000,0.026300,0.016400,0.000000,0.053300,0.206000,81.01100,132000.000000,3.00000
25%,73.250000,0.614750,0.651250,1.000000,-5.597750,0.000000,0.045225,0.174500,0.000000,0.108750,0.508250,104.76675,164555.000000,4.00000
50%,78.500000,0.699000,0.786500,5.000000,-4.385500,1.000000,0.077700,0.301500,0.000000,0.197000,0.659500,131.00400,177653.500000,4.00000
75%,81.000000,0.813000,0.842500,8.000000,-3.328500,1.000000,0.127000,0.498000,0.000000,0.567250,0.838750,150.01625,203149.500000,4.00000
max,97.000000,0.901000,0.984000,11.000000,-1.624000,1.000000,0.404000,0.917000,0.006090,0.956000,0.967000,172.05900,577819.000000,5.00000


In [12]:
#wanted to find a list of all playlists in the Top 50 Charts by Country category, but such category does not exist in the API.
#alternate resolution: manually copy and paste the playlist ids from spotify
#alternate resolution: instead of comparing data from all countries, we can analze the GLOBAL TOP 50 playlist vs GLOBAL VIRAL 50 playlist

url = f"https://api.spotify.com/v1/browse/categories/charts/playlists"
headers = {'authorization': f"Bearer {access_token}"}
response = requests.get(url, headers=headers)
cate= response.json()
pprint(cate)

{'error': {'message': "Specified id doesn't exist", 'status': 404}}


In [13]:
#data.to_csv("top50us.csv", encoding='utf-8', index=False)

In [15]:
data.popularity.mean()

76.68

In [16]:
stats.ttest_ind(totalmean, population2, equal_var=False)

NameError: name 'population2' is not defined